# Competitive data analysis

## 1. Introduction

This notebook contains the project of the course by Geek University (https://geekbrains.ru, Faculty of Artificial Intelligence), which aims to demonstrate to students how they can take part in Kaggle competitions.


The task of the project is to predict the potential dept of the loan based on customer data from different sources (https://www.kaggle.com/c/geekbrains-competitive-data-analysis/overview).


__Metric:__ ROC-AUC


__Type of the task:__ binary classification 

## 2. Data description

There are two groups of data: __basic__ - for getting fit and predict, __additional__ - for making features. 

### Basic data:


__train.csv__ - application id + target

__test.csv__ - application id without target

### Additional data:

__applications_history.csv__ - client's previous applications 

__bki.csv__ - client's previous credits provided by other financial institutions 

__client_profile.csv__ - other information about clients 

__payments.csv__ - repayment history for the previously disbursed credits 

## 3. Importing libraries and data

In [1]:
from typing import List, Optional
from tqdm import tqdm

import numpy as np
import pandas as pd


import scipy.stats as st
from scipy.stats import probplot, ks_2samp
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.ensemble import RandomForestRegressor

from imblearn.under_sampling import RandomUnderSampler 
from collections import Counter

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
import missingno as msno

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

import lightgbm as lgb
import xgboost as xgb
%matplotlib inline

In [2]:
PATH = 'data/'

In [109]:
train = pd.read_csv(PATH+"train.csv")
test = pd.read_csv(PATH+"test.csv")
applications_history = pd.read_csv(PATH+"applications_history.csv")
bki = pd.read_csv(PATH+"bki.csv")
client_profile = pd.read_csv(PATH+"client_profile.csv")
payments = pd.read_csv(PATH+"payments.csv")


print("train.shape = {} rows, {} cols".format(*train.shape))
print("test.shape = {} rows, {} cols".format(*test.shape))
print("applications_history.shape = {} rows, {} cols".format(*applications_history.shape))
print("bki.shape = {} rows, {} cols".format(*bki.shape))
print("client_profile.shape = {} rows, {} cols".format(*client_profile.shape))
print("payments.shape = {} rows, {} cols".format(*payments.shape))

train.shape = 110093 rows, 3 cols
test.shape = 165141 rows, 2 cols
applications_history.shape = 1670214 rows, 26 cols
bki.shape = 945234 rows, 17 cols
client_profile.shape = 250000 rows, 24 cols
payments.shape = 1023932 rows, 8 cols


## 4. Exploration data analysis

## 5. Solution strategy

As the purpose of the task is to achieve a high ROC-AUC score, it is important to comprehend what features are good or bad and what operations with data are useful (increase or decrease the score). For starting the process of the solution, checking and comparison results with different features I have decided to use LightGBM classifier with an internal categorical features preprocessor.


I'm going to get the first result using basic data (train.csv + client_profile) and compare it with other results through dropping and adding features by one. Since LightGBM is fast enough, it is possible to refit the model every time and use it as a validation 5-fold cross-validation without spending much time and resources.


Additionally, I will use XGBoost classifier encoding categorical features for stacking with LightGBM. 

## 6. Solution

### 6.1 First result

In [110]:
#join train.csv and client_profile.csv
train = pd.merge(train, client_profile, how='left', on = 'APPLICATION_NUMBER')

In [111]:
#join test.csv and client_profile.csv
test = pd.merge(test, client_profile, how='left', on = 'APPLICATION_NUMBER')

In [112]:
#for making categorical features for LightGBM categorical features preprocessor 
def cat_featutes(data):
  cat_features = list(data.select_dtypes(include=[np.object]).columns)
  cat_features = cat_features + ['AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT']
  return cat_features

def cat_featutes_maker(data, cat_features):    
  data[cat_features] = data[cat_features].astype('category')
  return data

In [113]:
cat_features = cat_featutes(train)

In [114]:
train = cat_featutes_maker(train, cat_features)
test = cat_featutes_maker(test, cat_features)

In [115]:
#this is the best set of hyper parameters from the teacher of the course :)
params = {
    'boosting_type': 'gbdt',
    'n_estimators': 5555,
    'learning_rate': 0.005134,
    'num_leaves': 54,
    'max_depth': 10,
    'subsample_for_bin': 240000,
    'reg_alpha': 0.436193,
    'reg_lambda': 0.479169,
    'colsample_bytree': 0.508716,
    'min_split_gain': 0.024766,
    'subsample': 0.7,
    'is_unbalance': False,
    'random_state': 27,
    'silent': -1,
    'verbose': -1
}

In [92]:
model = lgb.LGBMClassifier(**params)

In [93]:
#5-fold cross-validation
def make_cross_validation(X: pd.DataFrame,
                          y: pd.Series,
                          estimator: object,
                          metric: callable,
                          cv_strategy):
    """
    Parameters
    ----------
    X: pd.DataFrame

    y: pd.Series

    estimator: callable

    metric: callable

    cv_strategy: cross-validation generator
        
        KFold or StratifiedKFold.

    Returns
    -------
    oof_score: float
        OOF-predictions metric.

    fold_train_scores: List[float]
        OOF-predictions metric on each train fold

    fold_valid_scores: List[float]
        OOF-predictions metric on each valid fold
        
    oof_predictions: np.array
        OOF-predictions

    """
    estimators, fold_train_scores, fold_valid_scores = [], [], []
    oof_predictions = np.zeros(X.shape[0])

    for fold_number, (train_idx, valid_idx) in enumerate(cv_strategy.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]

        estimator.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)], early_stopping_rounds=500, eval_metric="auc", verbose=-1)
        y_train_pred = estimator.predict_proba(x_train)[:,1]
        y_valid_pred = estimator.predict_proba(x_valid)[:,1]

        fold_train_scores.append(metric(y_train, y_train_pred))
        fold_valid_scores.append(metric(y_valid, y_valid_pred))
        oof_predictions[valid_idx] = y_valid_pred

        msg = (
            f"Fold: {fold_number+1}, train-observations = {len(train_idx)}, "
            f"valid-observations = {len(valid_idx)}\n"
            f"train-score = {round(fold_train_scores[fold_number], 4)}, "
            f"valid-score = {round(fold_valid_scores[fold_number], 4)}, "
            f"target_median = {np.median(y_valid_pred)}"
        )
        print(msg)
        print("="*69)
        estimators.append(estimator)

    oof_score = metric(y, oof_predictions)
    print(f"CV-results train: {round(np.mean(fold_train_scores), 4)} +/- {round(np.std(fold_train_scores), 3)}")
    print(f"CV-results valid: {round(np.mean(fold_valid_scores), 4)} +/- {round(np.std(fold_valid_scores), 3)}")
    print(f"OOF-score = {round(oof_score, 4)}")

    return estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions

In [12]:
%%time
cv_strategy = KFold(n_splits=5, random_state=1)

estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
    train.drop(["TARGET"], axis=1), train["TARGET"], model, metric=roc_auc_score, cv_strategy=cv_strategy
)

/home/vavpavlov/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(
/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[1449]	training's auc: 0.821575	training's binary_logloss: 0.227792	valid_1's auc: 0.720861	valid_1's binary_logloss: 0.258537
Fold: 1, train-observations = 88074, valid-observations = 22019
train-score = 0.8216, valid-score = 0.7209, target_median = 0.06634254136008258


/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[1266]	training's auc: 0.812883	training's binary_logloss: 0.22891	valid_1's auc: 0.716222	valid_1's binary_logloss: 0.263484
Fold: 2, train-observations = 88074, valid-observations = 22019
train-score = 0.8129, valid-score = 0.7162, target_median = 0.06590021240429883


/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[1373]	training's auc: 0.81646	training's binary_logloss: 0.230285	valid_1's auc: 0.727456	valid_1's binary_logloss: 0.252346
Fold: 3, train-observations = 88074, valid-observations = 22019
train-score = 0.8165, valid-score = 0.7275, target_median = 0.06782127933455603


/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[1556]	training's auc: 0.826173	training's binary_logloss: 0.226751	valid_1's auc: 0.72011	valid_1's binary_logloss: 0.254166
Fold: 4, train-observations = 88075, valid-observations = 22018
train-score = 0.8262, valid-score = 0.7201, target_median = 0.06520441579779819


/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[1097]	training's auc: 0.804073	training's binary_logloss: 0.236064	valid_1's auc: 0.727921	valid_1's binary_logloss: 0.244994
Fold: 5, train-observations = 88075, valid-observations = 22018
train-score = 0.8041, valid-score = 0.7279, target_median = 0.06991997992573797
CV-results train: 0.8162 +/- 0.008
CV-results valid: 0.7225 +/- 0.005
OOF-score = 0.7216
CPU times: user 10min 3s, sys: 2.73 s, total: 10min 6s
Wall time: 2min 48s


In [13]:
basic_oof_score = oof_score
basic_oof_score

0.7215856282876134

### 6.2 Importance of existing features

In [14]:
#DF for results
scores = pd.DataFrame(columns=["feature", "oof_score", "diff_with_basic", "useful"])

In [16]:
features = list(train.columns)
features.remove('TARGET')

In [17]:
#results without each existing feature 
%%time

for feature in features:
    print (f"Without {feature}")
    tr = train.drop([feature], axis=1)
    cv_strategy = KFold(n_splits=5, random_state=1)

    estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
    tr.drop(["TARGET"], axis=1), tr["TARGET"], model, metric=roc_auc_score, cv_strategy=cv_strategy)
    
    diff_with_basic = round(basic_oof_score - oof_score, 4)
    
    useful = ''
    
    if diff_with_basic > 0:
        useful = 'YES'
    else:
        useful = 'NO'
    
    new_row = {"feature": feature, "oof_score": oof_score, "diff_with_basic": diff_with_basic, "useful": useful} 
    scores = scores.append(new_row, ignore_index=True)
    
    
    print ("*"*69)

Without APPLICATION_NUMBER


/home/vavpavlov/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(
/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[1351]	training's auc: 0.812627	training's binary_logloss: 0.230182	valid_1's auc: 0.722294	valid_1's binary_logloss: 0.25842
Fold: 1, train-observations = 88074, valid-observations = 22019
train-score = 0.8126, valid-score = 0.7223, target_median = 0.0664727617262303


/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[1527]	training's auc: 0.818347	training's binary_logloss: 0.226578	valid_1's auc: 0.718854	valid_1's binary_logloss: 0.263167
Fold: 2, train-observations = 88074, valid-observations = 22019
train-score = 0.8183, valid-score = 0.7189, target_median = 0.06584059404491413


...

Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[1152]	training's auc: 0.80391	training's binary_logloss: 0.234102	valid_1's auc: 0.727531	valid_1's binary_logloss: 0.252402
Fold: 3, train-observations = 88074, valid-observations = 22019
train-score = 0.8039, valid-score = 0.7275, target_median = 0.06813130199247623


/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[1594]	training's auc: 0.824824	training's binary_logloss: 0.227132	valid_1's auc: 0.719381	valid_1's binary_logloss: 0.254204
Fold: 4, train-observations = 88075, valid-observations = 22018
train-score = 0.8248, valid-score = 0.7194, target_median = 0.06517929920357468


/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[860]	training's auc: 0.787858	training's binary_logloss: 0.240804	valid_1's auc: 0.726562	valid_1's binary_logloss: 0.245452
Fold: 5, train-observations = 88075, valid-observations = 22018
train-score = 0.7879, valid-score = 0.7266, target_median = 0.07082871095121657
CV-results train: 0.8148 +/- 0.019
CV-results valid: 0.7222 +/- 0.004
OOF-score = 0.7209
*********************************************************************
CPU times: user 4h 1min 54s, sys: 47.4 s, total: 4h 2min 41s
Wall time: 1h 4min 45s


In [18]:
scores

,feature,oof_score,diff_with_basic,useful
0,APPLICATION_NUMBER,0.722438,-0.0009,NO
1,NAME_CONTRACT_TYPE,0.718877,0.0027,YES
2,GENDER,0.718944,0.0026,YES
3,CHILDRENS,0.721696,-0.0001,NO
4,TOTAL_SALARY,0.721375,0.0002,YES
5,AMOUNT_CREDIT,0.717975,0.0036,YES
6,AMOUNT_ANNUITY,0.719704,0.0019,YES
7,EDUCATION_LEVEL,0.717317,0.0043,YES
8,FAMILY_STATUS,0.721563,0.0000,NO
9,REGION_POPULATION,0.721180,0.0004,YES


### 6.3 Feature transformation

Here I'm introducing only important new features and data transformation, which bring good results through the same approach (5-fold cross-validation).

Honestly, I have made about 35 new features finally, however just several of them increased the score.

In [116]:
def feature_transformation(data):
    
    data['CREDIT_TIME'] = data['AMOUNT_CREDIT'] / data['AMOUNT_ANNUITY']
    data["DAYS_ON_LAST_JOB"] = data["DAYS_ON_LAST_JOB"].replace(365243, np.nan)
    
    data['EXTERNAL_SCORING_RATING_1'].fillna(1, inplace=True)
    data['EXTERNAL_SCORING_RATING_2'].fillna(1, inplace=True)
    data['EXTERNAL_SCORING_RATING_3'].fillna(1, inplace=True)
    
    #for function_name in ["min", "max", "mean", "nanmedian", "var"]:
    for function_name in ["mean"]:
        feature_name = "EXTERNAL_SCORING_RATING_{}".format(function_name)
        data[feature_name] = eval("np.{}".format(function_name))(
            data[["EXTERNAL_SCORING_RATING_1", "EXTERNAL_SCORING_RATING_2", "EXTERNAL_SCORING_RATING_3"]], axis=1
        )
    return data        

In [117]:
train = feature_transformation(train)

In [118]:
test = feature_transformation(test)

In [119]:
def bki_features(data, bki_data):
    
    bki_active = bki_data[bki_data['CREDIT_ACTIVE'] == 'Active']
    bki_active['diff_1'] = bki_active['AMT_CREDIT_SUM'] / bki_active['DAYS_CREDIT']
    
    apps_groupby = bki_active.groupby("APPLICATION_NUMBER", as_index=False)
    
    previous_app_counts = apps_groupby["diff_1"].sum()
    previous_app_counts = previous_app_counts.rename(columns={
        "PREV_APPLICATION_NUMBER": "PREV_APPS_COUNT"
    })
    
    unknown_customers = list(set(data['APPLICATION_NUMBER']) - set(client_profile['APPLICATION_NUMBER']))
    
    previous_app_counts = previous_app_counts.loc[~previous_app_counts['APPLICATION_NUMBER'].isin(unknown_customers)]
    
    data = pd.merge(data, previous_app_counts, how='left', on = 'APPLICATION_NUMBER')
    
    unknown_customers_2 = list(set(data['APPLICATION_NUMBER']) - set(client_profile['APPLICATION_NUMBER']) - set(previous_app_counts['APPLICATION_NUMBER']))
    
    data.loc[data['APPLICATION_NUMBER'].isin(unknown_customers_2), 'diff_1'] = 0
    
    return data
    

In [120]:
train = bki_features(train, bki)

<ipython-input-119-1187660770fb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bki_active['diff_1'] = bki_active['AMT_CREDIT_SUM'] / bki_active['DAYS_CREDIT']


In [121]:
test = bki_features(test, bki)

<ipython-input-119-1187660770fb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bki_active['diff_1'] = bki_active['AMT_CREDIT_SUM'] / bki_active['DAYS_CREDIT']


In [122]:
train.head()

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,...,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,CREDIT_TIME,EXTERNAL_SCORING_RATING_mean,diff_1
0,123687442,0,Cash,M,1.0,157500.0,855000.0,25128.0,Secondary / secondary special,Married,...,0.716570,0.0,0.0,1.0,0.0,0.0,2.0,34.025788,0.687756,NaN
1,123597908,1,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000
2,123526683,0,Cash,F,0.0,135000.0,1006920.0,42660.0,Higher education,Married,...,0.267869,0.0,0.0,0.0,7.0,0.0,4.0,23.603376,0.650006,4554.621849
3,123710391,1,Cash,M,0.0,180000.0,518562.0,22972.5,Secondary / secondary special,Married,...,0.170446,0.0,0.0,0.0,0.0,0.0,0.0,22.573164,0.447248,NaN
4,123590329,1,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000


In [123]:
test.head()

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,...,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,CREDIT_TIME,EXTERNAL_SCORING_RATING_mean,diff_1
0,123724268,Cash,M,0.0,117000.0,1125000.0,32895.0,Secondary / secondary special,Married,0.028663,...,1.000000,0.0,0.0,0.0,0.0,1.0,4.0,34.199726,0.876089,NaN
1,123456549,Cash,F,2.0,81000.0,312768.0,17095.5,Secondary / secondary special,Married,0.019689,...,0.188490,0.0,0.0,1.0,0.0,0.0,2.0,18.295341,0.588884,NaN
2,123428178,Credit Card,F,2.0,157500.0,450000.0,22500.0,Secondary / secondary special,Married,0.019101,...,0.382502,0.0,0.0,0.0,0.0,1.0,6.0,20.000000,0.511682,2700.000000
3,123619984,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000
4,123671104,Cash,F,1.0,90000.0,254700.0,24939.0,Higher education,Married,0.015221,...,0.415347,0.0,0.0,0.0,0.0,1.0,0.0,10.212920,0.546552,42724.734376


### 6.4 Final datasets

In [124]:
# a final set of features
features = ['TARGET', 'NAME_CONTRACT_TYPE',
 'GENDER', 
 'EDUCATION_LEVEL',
 'FAMILY_STATUS', 
 'AGE',
 'DAYS_ON_LAST_JOB',
 'OWN_CAR_AGE',  
 'EXTERNAL_SCORING_RATING_1',
 'EXTERNAL_SCORING_RATING_2',
 'EXTERNAL_SCORING_RATING_3',
 'CREDIT_TIME',
 'AMOUNT_CREDIT',
 'AMOUNT_ANNUITY',
 'REGION_POPULATION',
 'AMT_REQ_CREDIT_BUREAU_HOUR',
 'AMT_REQ_CREDIT_BUREAU_DAY',
 'AMT_REQ_CREDIT_BUREAU_WEEK',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'AMT_REQ_CREDIT_BUREAU_QRT', 
 'EXTERNAL_SCORING_RATING_mean', 
 'diff_1', 
 ]

In [125]:
train = train[features]

In [126]:
features.remove('TARGET')

In [127]:
test_AN = test['APPLICATION_NUMBER']

In [128]:
test = test[features]

In [129]:
test.head()

,NAME_CONTRACT_TYPE,GENDER,EDUCATION_LEVEL,FAMILY_STATUS,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,...,AMOUNT_CREDIT,AMOUNT_ANNUITY,REGION_POPULATION,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,EXTERNAL_SCORING_RATING_mean,diff_1
0,Cash,M,Secondary / secondary special,Married,16007.0,2646.0,20.0,1.000000,0.628266,1.000000,...,1125000.0,32895.0,0.028663,0.0,0.0,0.0,0.0,1.0,0.876089,NaN
1,Cash,F,Secondary / secondary special,Married,10315.0,459.0,NaN,1.000000,0.578161,0.188490,...,312768.0,17095.5,0.019689,0.0,0.0,1.0,0.0,0.0,0.588884,NaN
2,Credit Card,F,Secondary / secondary special,Married,13016.0,977.0,NaN,1.000000,0.152544,0.382502,...,450000.0,22500.0,0.019101,0.0,0.0,0.0,0.0,1.0,0.511682,2700.000000
3,Cash,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000
4,Cash,F,Higher education,Married,17743.0,9258.0,NaN,0.718604,0.505704,0.415347,...,254700.0,24939.0,0.015221,0.0,0.0,0.0,0.0,1.0,0.546552,42724.734376


In [130]:
train.head()

,TARGET,NAME_CONTRACT_TYPE,GENDER,EDUCATION_LEVEL,FAMILY_STATUS,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,...,AMOUNT_CREDIT,AMOUNT_ANNUITY,REGION_POPULATION,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,EXTERNAL_SCORING_RATING_mean,diff_1
0,0,Cash,M,Secondary / secondary special,Married,15728.0,1719.0,11.0,0.700784,0.645914,...,855000.0,25128.0,0.019101,0.0,0.0,1.0,0.0,0.0,0.687756,NaN
1,1,Cash,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000
2,0,Cash,F,Higher education,Married,21557.0,3618.0,NaN,1.000000,0.682149,...,1006920.0,42660.0,0.026392,0.0,0.0,0.0,7.0,0.0,0.650006,4554.621849
3,1,Cash,M,Secondary / secondary special,Married,22338.0,NaN,NaN,1.000000,0.171299,...,518562.0,22972.5,0.031329,0.0,0.0,0.0,0.0,0.0,0.447248,NaN
4,1,Cash,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000


### 6.5 LightGBM 

__6.5.1 LGBM model with final features__

Before fitting on all train dataset and predicting on test dataset, I  find the average number of estimators per validation for the hyperparameters of the final model. 

In [131]:
cv_strategy = KFold(n_splits=5, random_state=1)

estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
    train.drop(["TARGET"], axis=1), train["TARGET"], model, metric=roc_auc_score, cv_strategy=cv_strategy
)

/home/vavpavlov/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(
/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[1593]	training's auc: 0.826169	training's binary_logloss: 0.22415	valid_1's auc: 0.732335	valid_1's binary_logloss: 0.256325
Fold: 1, train-observations = 88074, valid-observations = 22019
train-score = 0.8262, valid-score = 0.7323, target_median = 0.06431217116199961
Training until validation scores don't improve for 500 rounds


/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Early stopping, best iteration is:
[2120]	training's auc: 0.847243	training's binary_logloss: 0.216086	valid_1's auc: 0.729083	valid_1's binary_logloss: 0.260528
Fold: 2, train-observations = 88074, valid-observations = 22019
train-score = 0.8472, valid-score = 0.7291, target_median = 0.06270943347445858
Training until validation scores don't improve for 500 rounds


/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Early stopping, best iteration is:
[1245]	training's auc: 0.814281	training's binary_logloss: 0.230141	valid_1's auc: 0.739057	valid_1's binary_logloss: 0.249937
Fold: 3, train-observations = 88074, valid-observations = 22019
train-score = 0.8143, valid-score = 0.7391, target_median = 0.06542598300522556


/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[2216]	training's auc: 0.849925	training's binary_logloss: 0.216873	valid_1's auc: 0.729174	valid_1's binary_logloss: 0.251797
Fold: 4, train-observations = 88075, valid-observations = 22018
train-score = 0.8499, valid-score = 0.7292, target_median = 0.06262188921016779
Training until validation scores don't improve for 500 rounds


/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/vavpavlov/.local/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Early stopping, best iteration is:
[1603]	training's auc: 0.829598	training's binary_logloss: 0.226533	valid_1's auc: 0.74166	valid_1's binary_logloss: 0.242391
Fold: 5, train-observations = 88075, valid-observations = 22018
train-score = 0.8296, valid-score = 0.7417, target_median = 0.06620613036055613
CV-results train: 0.8334 +/- 0.013
CV-results valid: 0.7343 +/- 0.005
OOF-score = 0.7333


__6.5.2 LGBM model for predictions on a test dataset__

In [132]:
#average number of estimators per validation
np.mean([1593, 2120, 1245, 2216, 1603])

1755.4

In [138]:
params = {
    'boosting_type': 'gbdt',
    'n_estimators': 1755, #fixed n_estimators 
    'learning_rate': 0.005134,
    'num_leaves': 54,
    'max_depth': 10,
    'subsample_for_bin': 240000,
    'reg_alpha': 0.436193,
    'reg_lambda': 0.479169,
    'colsample_bytree': 0.508716,
    'min_split_gain': 0.024766,
    'subsample': 0.7,
    'is_unbalance': False,
    'random_state': 42,
    'silent': -1,
    'verbose': -1
}

In [139]:
x_train = train.drop(["TARGET"], axis=1)
y_train = train["TARGET"]

In [140]:
model = lgb.LGBMClassifier(**params)
model.fit(
    X=x_train,
    y=y_train,
    eval_set=[(x_train, y_train)],    
    eval_metric="auc",
    verbose=200
)

[50]	training's auc: 0.734748	training's binary_logloss: 0.271551
[100]	training's auc: 0.737772	training's binary_logloss: 0.265371
[150]	training's auc: 0.741419	training's binary_logloss: 0.261018
[200]	training's auc: 0.74439	training's binary_logloss: 0.25766
[250]	training's auc: 0.747636	training's binary_logloss: 0.254951
[300]	training's auc: 0.750704	training's binary_logloss: 0.25276
[350]	training's auc: 0.753871	training's binary_logloss: 0.25085
[400]	training's auc: 0.757061	training's binary_logloss: 0.249225
[450]	training's auc: 0.760281	training's binary_logloss: 0.247736
[500]	training's auc: 0.763952	training's binary_logloss: 0.246358
[550]	training's auc: 0.766909	training's binary_logloss: 0.245137
[600]	training's auc: 0.769855	training's binary_logloss: 0.243983
[650]	training's auc: 0.772681	training's binary_logloss: 0.242913
[700]	training's auc: 0.775506	training's binary_logloss: 0.241872
[750]	training's auc: 0.778229	training's binary_logloss: 0.240904


LGBMClassifier(colsample_bytree=0.508716, is_unbalance=False,
               learning_rate=0.005134, max_depth=10, min_split_gain=0.024766,
               n_estimators=1755, num_leaves=54, random_state=42,
               reg_alpha=0.436193, reg_lambda=0.479169, silent=-1,
               subsample=0.7, subsample_for_bin=240000, verbose=-1)

In [141]:
y_pred_test_lgbm = model.predict_proba(test)

In [142]:
y_pred_test_lgbm

array([[0.96537049, 0.03462951],
       [0.73901369, 0.26098631],
       [0.8603227 , 0.1396773 ],
       ...,
       [0.91480179, 0.08519821],
       [0.980853  , 0.019147  ],
       [0.95463667, 0.04536333]])

In [145]:
result_test_x = pd.DataFrame({
    "APPLICATION_NUMBER": test_AN,
    "TARGET": y_pred_test_lgbm[:,1]})
result_test_x.head()

,APPLICATION_NUMBER,TARGET
0,123724268,0.034630
1,123456549,0.260986
2,123428178,0.139677
3,123619984,0.085198
4,123671104,0.014274


In [146]:
filename = 'lgbm_only.csv'
result_test_x.to_csv(filename, index=None)

### 6.6 XGBoost

__6.6.1 XGBoost model with final features__

...The same for XGBoost...

In [147]:
#categorical to number
def tun_dataset(data):
  numerical_features = data.select_dtypes(include=[np.number])
  categorical_features = pd.get_dummies(data.select_dtypes(include=['category']), drop_first=True) 
  num_plus_cat = numerical_features.join(categorical_features)
  return num_plus_cat

In [148]:
train = tun_dataset(train)

In [149]:
test = tun_dataset(test)

In [150]:
params = {
    "objective": "binary:logistic",
    "booster": "gbtree",
    "eval_metric": "auc",
    "eta": "0.05",
    "max_depth": 6,
    "gamma": 10,
    "subsample": 0.85,
    "colsample_bytree": 0.7,
    "colsample_bylevel": 0.632,
    "min_child_weight": 30,
    "alpha": 0,
    "lambda": 0,
    "nthread": 6,
    "random_seed": 27,
    "n_estimators": 5555,
}

In [151]:
model = xgb.XGBClassifier(**params)

In [152]:
cv_strategy = KFold(n_splits=5, random_state=1)

estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
    train.drop(["TARGET"], axis=1), train["TARGET"], model, metric=roc_auc_score, cv_strategy=cv_strategy
)

/home/vavpavlov/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


[19:47:55] WARNING: ../src/learner.cc:516: 
Parameters: { random_seed } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-auc:0.65609	validation_1-auc:0.64318
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 500 rounds.
[1]	validation_0-auc:0.68945	validation_1-auc:0.67182
[2]	validation_0-auc:0.69617	validation_1-auc:0.68083
[3]	validation_0-auc:0.69949	validation_1-auc:0.68537
[4]	validation_0-auc:0.70189	validation_1-auc:0.68643
[5]	validation_0-auc:0.70498	validation_1-auc:0.68961
[6]	validation_0-auc:0.70923	validation_1-auc:0.69282
[7]	validation_0-auc:0.71116	validation_1-auc:0.69533
[8]	validation_0-auc:0.71289	validation_1-auc:0.69804
[9]	validation_0-auc:0.71

[140]	validation_0-auc:0.74804	validation_1-auc:0.72298
[141]	validation_0-auc:0.74814	validation_1-auc:0.72304
[142]	validation_0-auc:0.74813	validation_1-auc:0.72298
[143]	validation_0-auc:0.74823	validation_1-auc:0.72308
[144]	validation_0-auc:0.74823	validation_1-auc:0.72316
[145]	validation_0-auc:0.74833	validation_1-auc:0.72340
[146]	validation_0-auc:0.74890	validation_1-auc:0.72387
[147]	validation_0-auc:0.74890	validation_1-auc:0.72387
[148]	validation_0-auc:0.74891	validation_1-auc:0.72397
[149]	validation_0-auc:0.74891	validation_1-auc:0.72397
[150]	validation_0-auc:0.74890	validation_1-auc:0.72394
[151]	validation_0-auc:0.74905	validation_1-auc:0.72403
[152]	validation_0-auc:0.74905	validation_1-auc:0.72403
[153]	validation_0-auc:0.74916	validation_1-auc:0.72418
[154]	validation_0-auc:0.74918	validation_1-auc:0.72461
[155]	validation_0-auc:0.74913	validation_1-auc:0.72465
[156]	validation_0-auc:0.74914	validation_1-auc:0.72451
[157]	validation_0-auc:0.74914	validation_1-auc:

[287]	validation_0-auc:0.75616	validation_1-auc:0.72761
[288]	validation_0-auc:0.75616	validation_1-auc:0.72761
[289]	validation_0-auc:0.75616	validation_1-auc:0.72761
[290]	validation_0-auc:0.75641	validation_1-auc:0.72781
[291]	validation_0-auc:0.75641	validation_1-auc:0.72781
[292]	validation_0-auc:0.75681	validation_1-auc:0.72847
[293]	validation_0-auc:0.75681	validation_1-auc:0.72847
[294]	validation_0-auc:0.75681	validation_1-auc:0.72844
[295]	validation_0-auc:0.75682	validation_1-auc:0.72846
[296]	validation_0-auc:0.75682	validation_1-auc:0.72846
[297]	validation_0-auc:0.75684	validation_1-auc:0.72850
[298]	validation_0-auc:0.75684	validation_1-auc:0.72850
[299]	validation_0-auc:0.75689	validation_1-auc:0.72854
[300]	validation_0-auc:0.75689	validation_1-auc:0.72854
[301]	validation_0-auc:0.75689	validation_1-auc:0.72854
[302]	validation_0-auc:0.75689	validation_1-auc:0.72854
[303]	validation_0-auc:0.75689	validation_1-auc:0.72854
[304]	validation_0-auc:0.75689	validation_1-auc:

[434]	validation_0-auc:0.76025	validation_1-auc:0.72904
[435]	validation_0-auc:0.76031	validation_1-auc:0.72897
[436]	validation_0-auc:0.76049	validation_1-auc:0.72893
[437]	validation_0-auc:0.76049	validation_1-auc:0.72893
[438]	validation_0-auc:0.76046	validation_1-auc:0.72908
[439]	validation_0-auc:0.76041	validation_1-auc:0.72893
[440]	validation_0-auc:0.76042	validation_1-auc:0.72903
[441]	validation_0-auc:0.76042	validation_1-auc:0.72903
[442]	validation_0-auc:0.76042	validation_1-auc:0.72903
[443]	validation_0-auc:0.76042	validation_1-auc:0.72903
[444]	validation_0-auc:0.76047	validation_1-auc:0.72895
[445]	validation_0-auc:0.76047	validation_1-auc:0.72895
[446]	validation_0-auc:0.76055	validation_1-auc:0.72885
[447]	validation_0-auc:0.76055	validation_1-auc:0.72885
[448]	validation_0-auc:0.76055	validation_1-auc:0.72885
[449]	validation_0-auc:0.76055	validation_1-auc:0.72885
[450]	validation_0-auc:0.76055	validation_1-auc:0.72885
[451]	validation_0-auc:0.76055	validation_1-auc:

...

[628]	validation_0-auc:0.76419	validation_1-auc:0.74170
[629]	validation_0-auc:0.76419	validation_1-auc:0.74170
[630]	validation_0-auc:0.76419	validation_1-auc:0.74170
[631]	validation_0-auc:0.76424	validation_1-auc:0.74169
[632]	validation_0-auc:0.76424	validation_1-auc:0.74169
[633]	validation_0-auc:0.76424	validation_1-auc:0.74169
[634]	validation_0-auc:0.76424	validation_1-auc:0.74169
[635]	validation_0-auc:0.76424	validation_1-auc:0.74169
[636]	validation_0-auc:0.76424	validation_1-auc:0.74169
[637]	validation_0-auc:0.76424	validation_1-auc:0.74169
[638]	validation_0-auc:0.76432	validation_1-auc:0.74172
[639]	validation_0-auc:0.76438	validation_1-auc:0.74179
[640]	validation_0-auc:0.76447	validation_1-auc:0.74164
[641]	validation_0-auc:0.76446	validation_1-auc:0.74167
[642]	validation_0-auc:0.76457	validation_1-auc:0.74170
[643]	validation_0-auc:0.76457	validation_1-auc:0.74170
[644]	validation_0-auc:0.76457	validation_1-auc:0.74170
[645]	validation_0-auc:0.76457	validation_1-auc:

[775]	validation_0-auc:0.76656	validation_1-auc:0.74254
[776]	validation_0-auc:0.76656	validation_1-auc:0.74254
[777]	validation_0-auc:0.76680	validation_1-auc:0.74256
[778]	validation_0-auc:0.76680	validation_1-auc:0.74256
[779]	validation_0-auc:0.76680	validation_1-auc:0.74256
[780]	validation_0-auc:0.76680	validation_1-auc:0.74256
[781]	validation_0-auc:0.76680	validation_1-auc:0.74256
[782]	validation_0-auc:0.76680	validation_1-auc:0.74256
[783]	validation_0-auc:0.76680	validation_1-auc:0.74243
[784]	validation_0-auc:0.76685	validation_1-auc:0.74239
[785]	validation_0-auc:0.76685	validation_1-auc:0.74239
[786]	validation_0-auc:0.76697	validation_1-auc:0.74249
[787]	validation_0-auc:0.76697	validation_1-auc:0.74249
[788]	validation_0-auc:0.76699	validation_1-auc:0.74252
[789]	validation_0-auc:0.76704	validation_1-auc:0.74229
[790]	validation_0-auc:0.76704	validation_1-auc:0.74229
[791]	validation_0-auc:0.76704	validation_1-auc:0.74229
[792]	validation_0-auc:0.76704	validation_1-auc:

[922]	validation_0-auc:0.76813	validation_1-auc:0.74209
[923]	validation_0-auc:0.76813	validation_1-auc:0.74209
[924]	validation_0-auc:0.76821	validation_1-auc:0.74218
[925]	validation_0-auc:0.76821	validation_1-auc:0.74218
[926]	validation_0-auc:0.76819	validation_1-auc:0.74207
[927]	validation_0-auc:0.76819	validation_1-auc:0.74207
[928]	validation_0-auc:0.76819	validation_1-auc:0.74207
[929]	validation_0-auc:0.76819	validation_1-auc:0.74207
[930]	validation_0-auc:0.76819	validation_1-auc:0.74207
[931]	validation_0-auc:0.76819	validation_1-auc:0.74207
[932]	validation_0-auc:0.76819	validation_1-auc:0.74207
[933]	validation_0-auc:0.76818	validation_1-auc:0.74205
[934]	validation_0-auc:0.76818	validation_1-auc:0.74205
[935]	validation_0-auc:0.76818	validation_1-auc:0.74205
[936]	validation_0-auc:0.76818	validation_1-auc:0.74205
[937]	validation_0-auc:0.76818	validation_1-auc:0.74205
[938]	validation_0-auc:0.76818	validation_1-auc:0.74205
[939]	validation_0-auc:0.76818	validation_1-auc:

[1068]	validation_0-auc:0.76983	validation_1-auc:0.74198
[1069]	validation_0-auc:0.76983	validation_1-auc:0.74198
[1070]	validation_0-auc:0.76983	validation_1-auc:0.74198
[1071]	validation_0-auc:0.76989	validation_1-auc:0.74198
[1072]	validation_0-auc:0.76989	validation_1-auc:0.74198
[1073]	validation_0-auc:0.76989	validation_1-auc:0.74198
[1074]	validation_0-auc:0.76988	validation_1-auc:0.74198
[1075]	validation_0-auc:0.76988	validation_1-auc:0.74198
[1076]	validation_0-auc:0.76988	validation_1-auc:0.74198
[1077]	validation_0-auc:0.76988	validation_1-auc:0.74198
[1078]	validation_0-auc:0.76992	validation_1-auc:0.74189
[1079]	validation_0-auc:0.76992	validation_1-auc:0.74189
[1080]	validation_0-auc:0.76993	validation_1-auc:0.74194
[1081]	validation_0-auc:0.76993	validation_1-auc:0.74194
[1082]	validation_0-auc:0.76993	validation_1-auc:0.74194
[1083]	validation_0-auc:0.76993	validation_1-auc:0.74194
[1084]	validation_0-auc:0.76993	validation_1-auc:0.74194
[1085]	validation_0-auc:0.76993

[1212]	validation_0-auc:0.77154	validation_1-auc:0.74213
[1213]	validation_0-auc:0.77154	validation_1-auc:0.74213
[1214]	validation_0-auc:0.77154	validation_1-auc:0.74213
[1215]	validation_0-auc:0.77154	validation_1-auc:0.74213
[1216]	validation_0-auc:0.77154	validation_1-auc:0.74213
[1217]	validation_0-auc:0.77154	validation_1-auc:0.74213
[1218]	validation_0-auc:0.77154	validation_1-auc:0.74213
[1219]	validation_0-auc:0.77154	validation_1-auc:0.74213
[1220]	validation_0-auc:0.77154	validation_1-auc:0.74213
[1221]	validation_0-auc:0.77154	validation_1-auc:0.74213
[1222]	validation_0-auc:0.77154	validation_1-auc:0.74213
[1223]	validation_0-auc:0.77154	validation_1-auc:0.74213
[1224]	validation_0-auc:0.77154	validation_1-auc:0.74213
[1225]	validation_0-auc:0.77156	validation_1-auc:0.74212
[1226]	validation_0-auc:0.77156	validation_1-auc:0.74212
[1227]	validation_0-auc:0.77156	validation_1-auc:0.74212
[1228]	validation_0-auc:0.77156	validation_1-auc:0.74212
[1229]	validation_0-auc:0.77156

In [153]:
np.mean([1075, 884, 895, 745, 767])

873.2

In [154]:
x_train = train.drop(["TARGET"], axis=1)
y_train = train["TARGET"]

Unfortunatelly there aren't Exploratory Data Analysis the next two cells are needed to equalize the number of features in the train and test :/  

In [155]:
tt_features = list(set(list(test.columns)) - set(list(train.columns)))

In [156]:
tr_features = list(set(list(train.columns)) - set(list(test.columns))) 

In [157]:
train = train.drop(tr_features, axis=1)

In [158]:
test = test.drop(tt_features, axis=1)

In [159]:
params = {
    "objective": "binary:logistic",
    "booster": "gbtree",
    "eval_metric": "auc",
    "eta": "0.05",
    "max_depth": 6,
    "gamma": 10,
    "subsample": 0.85,
    "colsample_bytree": 0.7,
    "colsample_bylevel": 0.632,
    "min_child_weight": 30,
    "alpha": 0,
    "lambda": 0,
    "nthread": 6,
    "random_seed": 27,
    "n_estimators": 873,
}

In [160]:
model = xgb.XGBClassifier(**params)
model.fit(
    X=x_train,
    y=y_train,
    eval_set=[(x_train, y_train)],    
    eval_metric="auc",
    verbose=200
)

[20:01:30] WARNING: ../src/learner.cc:516: 
Parameters: { random_seed } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-auc:0.65332
[200]	validation_0-auc:0.75256
[400]	validation_0-auc:0.75880
[600]	validation_0-auc:0.76335
[800]	validation_0-auc:0.76624
[872]	validation_0-auc:0.76689


XGBClassifier(alpha=0, base_score=0.5, booster='gbtree',
              colsample_bylevel=0.632, colsample_bynode=1, colsample_bytree=0.7,
              eta='0.05', eval_metric='auc', gamma=10, gpu_id=-1,
              importance_type='gain', interaction_constraints='', lambda=0,
              learning_rate=0.0500000007, max_delta_step=0, max_depth=6,
              min_child_weight=30, missing=nan, monotone_constraints='()',
              n_estimators=873, n_jobs=6, nthread=6, num_parallel_tree=1,
              random_seed=27, random_state=0, reg_alpha=0, reg_lambda=0,
              scale_pos_weight=1, subsample=0.85, tree_method='exact', ...)

In [161]:
y_pred_test_xgb = model.predict_proba(test)

In [162]:
y_pred_test_xgb

array([[0.92899084, 0.07100917],
       [0.71526265, 0.28473738],
       [0.87644124, 0.12355877],
       ...,
       [0.9152941 , 0.08470587],
       [0.9814359 , 0.0185641 ],
       [0.9480052 , 0.05199481]], dtype=float32)

### 6.7 LightGBM + XGBoost

Counting the average of Lgbm and XGBoost predictions.

In [163]:
scores = pd.DataFrame({
    "lgbm": y_pred_test_lgbm[:,1],
    "xgb": y_pred_test_xgb[:,1]
})

In [164]:
scores.head(10)

,lgbm,xgb
0,0.034630,0.071009
1,0.260986,0.284737
2,0.139677,0.123559
3,0.085198,0.084706
4,0.014274,0.013599
5,0.031712,0.031053
6,0.057907,0.056952
7,0.100388,0.103192
8,0.008492,0.013319
9,0.021653,0.024609


In [165]:
scores_mean = scores.mean(axis=1)

In [166]:
result_test_x = pd.DataFrame({
    "APPLICATION_NUMBER": test_AN,
    "TARGET": scores_mean})
result_test_x.head()

,APPLICATION_NUMBER,TARGET
0,123724268,0.052819
1,123456549,0.272862
2,123428178,0.131618
3,123619984,0.084952
4,123671104,0.013937


In [167]:
filename = 'xgb+lgbm.csv'
result_test_x.to_csv(filename, index=None)
# the best Kaggle submission (0.735448)

## 7. Conclusions

In the process of this work, it turned out that new features don't always work, but it is also possible to increase the result quite well by dropping existing ones.
It is also very important to select the hyperparameters of the models and averaging the results of different models.